In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [2]:
def display(img, cmap='gray'):
    fig = plt.figure(figsize=(12,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap='gray')

In [3]:
road = cv2.imread('DATA/road_image.jpg')

In [4]:
road_copy = np.copy(road)

In [5]:
road.shape

(600, 800, 3)

In [6]:
marker_image = np.zeros(road.shape[:2], dtype=np.int32)

In [7]:
segments = np.zeros(road.shape, dtype=np.uint8)

In [8]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [9]:
colors = []
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[:3])*255)

In [10]:
for i in range(10):
    colors.append(create_rgb(i))

In [11]:
colors

[(31.0, 119.0, 180.0),
 (255.0, 127.0, 14.0),
 (44.0, 160.0, 44.0),
 (214.0, 39.0, 40.0),
 (148.0, 103.0, 189.0),
 (140.0, 86.0, 75.0),
 (227.0, 119.0, 194.0),
 (127.0, 127.0, 127.0),
 (188.0, 189.0, 34.0),
 (23.0, 190.0, 207.0)]

In [12]:
#Global variables
#Color choice
current_marker = 1
#markers updated by watershed
marks_updated = False

In [13]:
#callback function
def mouse_callback(event,x,y,flags,params):
    global marks_updated
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(marker_image, (x,y), 10, (current_marker), -1)
        cv2.circle(road_copy, (x,y), 10, colors[current_marker], -1)
        marks_updated = True

In [14]:
#While true
cv2.namedWindow('Road image')
cv2.setMouseCallback('Road image', mouse_callback)

while True: 
    cv2.imshow('Watershed Segments', segments)
    cv2.imshow('Road image', road_copy)
    #close windows
    k = cv2.waitKey(1)
    if k == 27:
        break
        
    #clear all colors
    elif k == ord('c'):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[:2], dtype=np.int32)
        segments = np.zeros(road.shape,dtype=np.uint8)
    #update color choice
    elif k > 0 and chr(k).isdigit():
        current_marker = int(chr(k))
    #update markings
    if marks_updated:
        marker_image_copy = marker_image.copy()
        cv2.watershed(road, marker_image_copy)
        segments = np.zeros(road.shape, dtype=np.uint8)
        for color_ind in range(10):
            segments[marker_image_copy==(color_ind)] = colors[color_ind]